In [1]:
#jupyterlab에서 django shell을 실행을 시키기 위한 설정 (shell: project 모듈들을 테스트 할 수 있는 파이썬 shell)

import os
import django

os.environ['DJANGO_SETTINGS_MODULE'] = "config.settings" #consfig/settings.py
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = "true"

django.setup()

In [3]:
from polls.models import Question, Choice

print(Question.objects.get(pk=1))

1. 좋아하는 꽃은 무엇인가요?


In [ ]:
# Model 클래스.objects : Model Manager - Model 클래스와 연결된 테이블로부터 SQL(select)문을 실행하는 매소드를 제공
# 
type(Question.objects), type(Choice.objects)

(django.db.models.manager.Manager, django.db.models.manager.Manager)

In [5]:
result = Question.objects.all() #select* from question;
type(result) # 조회결과들을 담는 타입(QuerySet)


django.db.models.query.QuerySet

In [ ]:
result[0] #subscriptable 타입

<Question: 1. 좋아하는 꽃은 무엇인가요?>

In [7]:
for q in result : #Iterable 반복가능
    print(q)

1. 좋아하는 꽃은 무엇인가요?
2. 좋아하는 동물은 무엇인가요?


In [9]:
print(result.query)

SELECT "polls_question"."id", "polls_question"."question_text", "polls_question"."pub_date" FROM "polls_question"


In [ ]:
# 조회
# all() : where 절 넣을 필요 없이 전체 행을 조회
# where 절 조건으로 조회
#filter(), exclude(), get()

In [12]:
from polls.models import Question, Choice # 모델 클래스들 improt

#전체 조회
result = Question.objects.all()
for q in result:
 print(q, type(q))

1. 좋아하는 꽃은 무엇인가요? <class 'polls.models.Question'>
2. 좋아하는 동물은 무엇인가요? <class 'polls.models.Question'>
3. 좋아하는 소설 장르는 무엇인가요? <class 'polls.models.Question'>


In [ ]:
for q in result:
 print(q.id, q.question_text, q.pub_date, sep=", ")
 #모델의 primary key Field의 값을 조회할때: 모델명.pk, 모델.field명

1,좋아하는 꽃은 무엇인가요?,2026-01-15 02:32:19.296818+00:00
2,좋아하는 동물은 무엇인가요?,2026-01-15 02:36:26.847505+00:00
3,좋아하는 소설 장르는 무엇인가요?,2026-01-15 03:50:03.024682+00:00


In [17]:
#indexing
result[0]

<Question: 1. 좋아하는 꽃은 무엇인가요?>

In [20]:
try:
    Question.objects.get(id=10) 
except:
     print("조회결과가 없습니다.")

조회결과가 없습니다.


In [ ]:
Choice.objects.get(votes=0) #where_votes =0
except print("조회결과가 1개 이상입니다.")


MultipleObjectsReturned: get() returned more than one Choice -- it returned 3!

In [23]:
result = Choice.objects.filter(votes=0)
for c in result:
    print(c.id, c.pk, c.choice_text, c.votes, c.question_id, c.question)
    #(c.question) : 참조 모델 객체, c.question_id (모델 이름pk dlfma): 참조모델의 pk 값

6 6 토끼 0 2 2. 좋아하는 동물은 무엇인가요?
9 9 추리 0 3 3. 좋아하는 소설 장르는 무엇인가요?
12 12 비문학 0 3 3. 좋아하는 소설 장르는 무엇인가요?


In [24]:
result = Question.objects.exclude(pk=1)
for r in result:
    print(r)

2. 좋아하는 동물은 무엇인가요?
3. 좋아하는 소설 장르는 무엇인가요?


In [28]:
# field 조건
# field 이름__연산자 = 비교값

result = Choice.objects.filter(votes=0) #where votes = 0
result = Choice.objects.filter(votes__lt=20) # where votes<20
result = Choice.objects.filter(votes__lte=20) # where votes=<20
result = Choice.objects.filter(votes__gt=20) # where votes>20
result = Choice.objects.filter(votes__gte=20) # where votes=>20

for q in result :
     print(q.id, q.choice_text, q.votes, sep=", ")

print(result.query)

4, 고양이, 20
7, 기린, 30
10, 한국 문학, 25
11, 판타지, 50
13, 오리, 42
15, 프레지아, 30
SELECT "polls_choice"."id", "polls_choice"."choice_text", "polls_choice"."votes", "polls_choice"."question_id" FROM "polls_choice" WHERE "polls_choice"."votes" >= 20


In [ ]:
# 문자열 부분일치 조회
#where question_text like '좋아하는%'
result = Question.objects.filter(question_text__startswith="좋아하는")

#where not question_text like '좋아하는%'
result = Question.objects.exclude(question_text__startswith="좋아하는")

#where question_text like '%무엇인가요?'
result = Question.objects.filter(question_text__endswith="무엇인가요?")

#where question_text like '%장르%'
result = Question.objects.filter(question_text__contains="장르")

for q in result :
     print(q.id, q.question_text, q.pub_date.strftime('%Y-%m-%d'), sep=", ")

print(result.query)

3, 좋아하는 소설 장르는 무엇인가요?, 2026-01-15
SELECT "polls_question"."id", "polls_question"."question_text", "polls_question"."pub_date" FROM "polls_question" WHERE "polls_question"."question_text" LIKE %장르% ESCAPE '\'


In [40]:
# IN, between
# where id in [1, 3, 5]
result = Choice.objects.filter(pk__in=[1, 3, 5])

# where votesbetween 10 and 30
result = Choice.objects.filter(votes__range=[10, 30])

for c in result:
    print(c, c.votes)

print(result.query)

3. 강아지 10
4. 고양이 20
7. 기린 30
8. 코끼리 15
10. 한국 문학 25
14. 민들레 10
15. 프레지아 30
SELECT "polls_choice"."id", "polls_choice"."choice_text", "polls_choice"."votes", "polls_choice"."question_id" FROM "polls_choice" WHERE "polls_choice"."votes" BETWEEN 10 AND 30


In [41]:
# 조건이 여러개인 경우 - AND 

# where id >= 5 and votes < 20
result = Choice.objects.filter(
    pk__gte=5,
    votes__lt=20
)

for c in result:
    print(c, c.votes)

print(result.query)


5. 백합 2
6. 토끼 0
8. 코끼리 15
9. 추리 0
12. 비문학 0
14. 민들레 10
SELECT "polls_choice"."id", "polls_choice"."choice_text", "polls_choice"."votes", "polls_choice"."question_id" FROM "polls_choice" WHERE ("polls_choice"."id" >= 5 AND "polls_choice"."votes" < 20)


In [43]:
# OR - Q 객체에 조건들을 넣어준다. Q(조건)|Q(조건)|....
from django.db.models import Q

# where votes >= 20 or id <=10
result = Choice.objects.filter(
    Q(votes__gte=20)| Q(pk__lte=10)
)

# where not (votes >= 20) or id <=10
result = Choice.objects.filter(
    ~Q(votes__gte=20)| Q(pk__lte=10)
)

for c in result:
    print(c, c.votes)

print(result.query)

1. 개나리 5
2. 장미 5
3. 강아지 10
4. 고양이 20
5. 백합 2
6. 토끼 0
7. 기린 30
8. 코끼리 15
9. 추리 0
10. 한국 문학 25
12. 비문학 0
14. 민들레 10
SELECT "polls_choice"."id", "polls_choice"."choice_text", "polls_choice"."votes", "polls_choice"."question_id" FROM "polls_choice" WHERE (NOT ("polls_choice"."votes" >= 20) OR "polls_choice"."id" <= 10)


In [46]:
# 정렬 - QuerySet.order_by("컬럼명","컬럼명") "-컬럼명":내림차순(desc)
#order by votes asc id desc
result = Choice.objects.all().order_by("votes", "-id")

for c in result:
    print(c, c.votes)

print(result.query)

12. 비문학 0
9. 추리 0
6. 토끼 0
5. 백합 2
2. 장미 5
1. 개나리 5
14. 민들레 10
3. 강아지 10
8. 코끼리 15
4. 고양이 20
10. 한국 문학 25
15. 프레지아 30
7. 기린 30
13. 오리 42
11. 판타지 50
SELECT "polls_choice"."id", "polls_choice"."choice_text", "polls_choice"."votes", "polls_choice"."question_id" FROM "polls_choice" ORDER BY "polls_choice"."votes" ASC, "polls_choice"."id" DESC


In [ ]:
# 특정 컬럼들만 조회
result = Choice.objects.all().values("pk","choice_text")
print(type(result))

for c in result:
    print(type(c), c) # 개별 조회 결과를 dictionary 로 반환

<class 'django.db.models.query.QuerySet'>
<class 'dict'> {'pk': 1, 'choice_text': '개나리'}
<class 'dict'> {'pk': 2, 'choice_text': '장미'}
<class 'dict'> {'pk': 3, 'choice_text': '강아지'}
<class 'dict'> {'pk': 4, 'choice_text': '고양이'}
<class 'dict'> {'pk': 5, 'choice_text': '백합'}
<class 'dict'> {'pk': 6, 'choice_text': '토끼'}
<class 'dict'> {'pk': 7, 'choice_text': '기린'}
<class 'dict'> {'pk': 8, 'choice_text': '코끼리'}
<class 'dict'> {'pk': 9, 'choice_text': '추리'}
<class 'dict'> {'pk': 10, 'choice_text': '한국 문학'}
<class 'dict'> {'pk': 11, 'choice_text': '판타지'}
<class 'dict'> {'pk': 12, 'choice_text': '비문학'}
<class 'dict'> {'pk': 13, 'choice_text': '오리'}
<class 'dict'> {'pk': 14, 'choice_text': '민들레'}
<class 'dict'> {'pk': 15, 'choice_text': '프레지아'}


In [53]:
# 집계

# aggregate(집계함수객체(컬럼명), 집계함수객체(컬럼명),...) -> 전체 행 기준 집계
# values('groupby 기준칼럼').annotate(집계함수객체(컬럼명), ...) -> groupby 후 집계


from django.db import models

#select count(id) from choice;
result = Choice.objects.aggregate(
    models.Count("id"),
    models.Avg("votes")
)

#결과 key 를 지정
result = Choice.objects.aggregate(
   cnt= models.Count("id"),
   avg= models.Avg("votes")
)


result

{'cnt': 15, 'avg': 16.266666666666666}

In [54]:
result['cnt']

15

In [56]:
# Group by 집계

#select min(vote), max(vote)      from choice groupby question
result = Choice.objects.values("question").annotate(
    models.Min("votes"), models.Max("votes")
)

print(type(result))
for r in result:
    print(r)

<class 'django.db.models.query.QuerySet'>
{'question': 1, 'votes__min': 2, 'votes__max': 30}
{'question': 2, 'votes__min': 0, 'votes__max': 42}
{'question': 3, 'votes__min': 0, 'votes__max': 50}


In [ ]:
# Join

#자식 모델 조회
#자식 모델을 기준으로 참조 부모 객체를 조회 -> foringnkey 속성으로 조회.

choice = Choice.objects.get(pk=1) #모델 반환
#Choice.question 필드 -> Foreingnkey 필드 (Question ckawh)
question = choice.question
print(type(question))
question

<class 'polls.models.Question'>


<Question: 1. 좋아하는 꽃은 무엇인가요?>

In [61]:
result = Choice.objects.filter(choice_text__contains='리')

for c in result:
    print(c.choice_text, c.question.question_text, sep=", ")

개나리, 좋아하는 꽃은 무엇인가요?
코끼리, 좋아하는 동물은 무엇인가요?
추리, 좋아하는 소설 장르는 무엇인가요?
오리, 좋아하는 동물은 무엇인가요?


In [ ]:
# 부모 테이블 조회 결과를 이용해서 자식 테이블 참조 값들을 조회
## 부모Model객체.relate_name : 부모 Model 객체를 참조하는 자식의 행들로 부터 조회할 수 있는 ModelManager가 반환됨
#
#releated_name : 자식모델클래스이름(소문자)_set, ForeignKey(...., related_name="related")

question = Question.objects.get(pk=1)
choice_list = question.choice_set.all() # all 대상: question(pk=1)을 참조하는 choice 조회대상.

print("질문:", question.question_text)
for idx, ch in enumerate(choice_list, start=1):
    print(idx, ch.choice_text)

질문: 좋아하는 꽃은 무엇인가요?
1 개나리
2 장미
3 백합
4 민들레
5 프레지아


In [68]:
# Insert/Update

#모델 객체를 생성  - Initalixer에 keyword 인자로 Filed 값을 대입.
new_question = Question(question_text="새로운 질문입니다.")
print(new_question.pk)

new_question.save() #insert

None


In [73]:
q = Question.objects.get(pk=2)
q.question_text = "당신이 싫어하는 동물은 무엇인가요?"
print(q.pk, q.question_text)
q.save() # pk=2인 행을 update

2 당신이 싫어하는 동물은 무엇인가요?


In [74]:
result = Question.objects.all()
for q in result:
    print(q)

1. 좋아하는 꽃은 무엇인가요?
2. 당신이 싫어하는 동물은 무엇인가요?
3. 좋아하는 소설 장르는 무엇인가요?
4. 새로운 질문입니다.


In [75]:
# 삭제
del_q = Question(pk=4)
del_q.delete() #모델의 pk rkqtdmf tkrwp

(1, {'polls.Question': 1})

In [76]:
result = Question.objects.all()
for q in result:
    print(q)

1. 좋아하는 꽃은 무엇인가요?
2. 당신이 싫어하는 동물은 무엇인가요?
3. 좋아하는 소설 장르는 무엇인가요?


In [80]:
#삭제 전 개수
cont1 = Choice.objects.aggregate(models.Count("id"))
cont1

{'id__count': 15}

In [81]:
# delete from tb where 삭제 조건 -> filter로 삭제할 조건의 모델을 조회 후에 삭제
#delete from choice where votes <10

del_list = Choice.objects.filter(votes__lt=10)
for c in del_list:
    # print(c)
    c.delete()

In [82]:
cont1 = Choice.objects.aggregate(models.Count("id"))
cont1

{'id__count': 9}